### 1. Purpose of this notebook

The purpose of this notebook is to pre process the data, including removing the data not necessary for the analysis, inclusion of new data and some dtypes transformations.

We removed seven customers from the base. Six had the value of purchases with no value - although there are values in the column MntTotal - and one customer for having more purchases at a discount than the total purchases made.

### 2. Read data

#### 2.1 Import Python packages

In [2]:
import pandas as pd
import datetime

from src.paths import DATA

In [3]:
df = pd.read_csv(DATA / 'ml_project1_data_cleaned.csv')

df = (df.assign(Dt_Customer = pd.to_datetime(df['Dt_Customer'])))

### 3. New columns

For our analysis, we need some new columns:

- The age for each customer.
- The total amount spent in the last two years.
- The total number of purchases in the three channels in the last two years.
- The total of accepteds the offer in the first five campaigns.
- The total of accepteds the offer in the first five campaigns and the pilot campaigns.
- The total of year after registration.
- The total children.

#### Age

In [4]:
current_year = datetime.date.today().year

df = (df
      .assign(Age = current_year - df['Year_Birth'])
     )

#### Total amount spent

In [5]:
df = (df
      .assign(MntTotal = df[['MntWines', 'MntFruits','MntMeatProducts',
                              'MntFishProducts', 'MntSweetProducts']]
                          .sum(axis=1))
     )

#### Total purchases

In [6]:
df = (df
      .assign(NumTotalPurchases = df[['NumWebPurchases','NumCatalogPurchases',
                                     'NumStorePurchases',]]
                                  .sum(axis=1))
     )

#### Total accepted in the first five campaigns

In [7]:
df = (df
      .assign(AcceptedTotalFirstFiveCmps = df[['AcceptedCmp3', 'AcceptedCmp4', 
                                               'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2']]
                                          .sum(axis=1))
     )

#### Total accepted in the first five campaigns + pilot campaign 

In [8]:
df = (df
      .assign(AcceptedTotalFirstFiveCmpsMorePilot = df[['AcceptedTotalFirstFiveCmps', 'Response']]
                          .sum(axis=1))
     )

#### Total year after registration

In [9]:
df = (df
      .assign(TotalYearAfterRegistration = current_year - df.Dt_Customer.dt.year)
     )

#### Total of childrens

In [10]:
df = (df
      .assign(TotalChildren = df['Kidhome'] + df['Teenhome'])
     )

#### Results

In [11]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
ID,2240.0,5592.159821,3246.662198,0.0,2828.25,5458.5,8427.75,11191.0
Year_Birth,2240.0,1968.805804,11.984069,1893.0,1959.00,1970.0,1977.00,1996.0
Income,2216.0,52247.251354,25173.076661,1730.0,35303.00,51381.5,68522.00,666666.0
Kidhome,2240.0,0.444196,0.538398,0.0,0.00,0.0,1.00,2.0
Teenhome,2240.0,0.506250,0.544538,0.0,0.00,0.0,1.00,2.0
Recency,2240.0,49.109375,28.962453,0.0,24.00,49.0,74.00,99.0
MntWines,2240.0,303.935714,336.597393,0.0,23.75,173.5,504.25,1493.0
MntFruits,2240.0,26.302232,39.773434,0.0,1.00,8.0,33.00,199.0
MntMeatProducts,2240.0,166.950000,225.715373,0.0,16.00,67.0,232.00,1725.0
MntFishProducts,2240.0,37.525446,54.628979,0.0,3.00,12.0,50.00,259.0


There are some customers without purchases (NumTotalPurchases), but with value in MntTotal.
It's necessary remove them, as they may be errors.

In [12]:
df.query('MntTotal > 0 and NumTotalPurchases == 0')['ID']

655      5555
981      3955
1245     6862
1524    11110
1846     9931
2132    11181
Name: ID, dtype: int64

We will also remove cases of customers with more discounts than purchases:

In [13]:
df.query(' NumDealsPurchases > NumTotalPurchases')['ID']

1042    10749
1245     6862
1846     9931
Name: ID, dtype: int64

In [14]:
df.drop(df.query("MntTotal > 0 and NumTotalPurchases == 0").index, inplace=True)
df.drop(df.query("NumDealsPurchases > NumTotalPurchases").index, inplace=True)
df.describe().T

,count,mean,std,min,25%,50%,75%,max
ID,2233.0,5583.114644,3244.053607,0.0,2815.0,5453.0,8418.0,11191.0
Year_Birth,2233.0,1968.803403,11.984284,1893.0,1959.0,1970.0,1977.0,1996.0
Income,2209.0,52262.071978,24917.065825,2447.0,35416.0,51411.0,68487.0,666666.0
Kidhome,2233.0,0.444693,0.538575,0.0,0.0,0.0,1.0,2.0
Teenhome,2233.0,0.507389,0.544658,0.0,0.0,0.0,1.0,2.0
Recency,2233.0,49.085087,28.976738,0.0,24.0,49.0,74.0,99.0
MntWines,2233.0,304.849978,336.724809,0.0,24.0,176.0,505.0,1493.0
MntFruits,2233.0,26.374384,39.813403,0.0,2.0,8.0,33.0,199.0
MntMeatProducts,2233.0,167.439767,225.895591,0.0,16.0,68.0,232.0,1725.0
MntFishProducts,2233.0,37.637707,54.677557,0.0,3.0,12.0,50.0,259.0


### 4. Remove columns

- The column Z_Revenue is wrong. All the rows containing the same valor: 11. We know that the sucess of the oilot campaign was 15%, not 100%. We created a new column to fix it.

#### Z_Revenue

In [15]:
df = df.drop(columns='Z_Revenue')

### 5. Data transformation

In [16]:
columns = ['AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
           'AcceptedCmp2', 'Complain', 'Response']

df[columns] = df[columns].astype(bool)

In [17]:
df.to_csv(DATA / 'ml_project1_data_pre_processed.csv', index=False)